In [4]:
from mistralai.models.chat_completion import ChatMessage
from mistralai.client import MistralClient
#from langchain_mistralai import ChatMistralAI

MISTRAL_API_KEY = "mgNvjI0hi3WcPdMvPMVgy8LKQFQhVtBx"  # Replace with your Mistral API key


client = MistralClient(api_key=MISTRAL_API_KEY)

In [5]:
client

In [6]:
messages = [
    ChatMessage(role="user", content="What is the capital of France?")
]

chat_response = client.chat(
    model="mistral-small",
    messages=messages
)

print(chat_response.choices[0].message.content)



MistralAPIException: Cannot stream response. Status: 400